Note: This notebook assumes that "gmn_extract_candidate_contrails.ipynb" and "gmn_run_segmentation.ipynb" has been run.

Here we visualise the outputs of the previously ran scripts.

In [1]:
import sys
sys.path.append("..")

import os
from glob import glob
import json
from PIL import Image, ImageDraw
import imageio

In [2]:
base_dir = '../data/gmn_extracted_flight_images/US001N'
circle_radius = 5

In [3]:
meta_data_files = glob(os.path.join(base_dir, '*/*/metadata/*.json'))

for meta_data_file in meta_data_files:
    images = []

    flight_dir = os.path.dirname(os.path.dirname(meta_data_file))
    output_video_dir = os.path.join(flight_dir, 'debug')
    os.makedirs(output_video_dir, exist_ok=True)

    with open(meta_data_file, 'r') as f:
        data = json.load(f)

    # Visualise frames with flight
    for frame in data['frames_with_flight']:
        image = Image.open(frame['file_path']).convert('RGBA')
        draw  = ImageDraw.Draw(image)

        for x, y in zip(frame['advected_x_coords'], frame['advected_y_coords']):
            draw.ellipse([(x-circle_radius, y-circle_radius),
                            (x+circle_radius, y+circle_radius)],
                            fill=(0, 255, 0), outline=(0, 255, 0))

        draw.ellipse([(frame['x_coord']-circle_radius, frame['y_coord']-circle_radius),
                        (frame['x_coord']+circle_radius, frame['y_coord']+circle_radius)],
                        fill=(255, 0, 0), outline=(255, 0, 0))
        
        images.append(image)

    # Visualise frames after flight
    sam2_output = sorted(glob(os.path.join(flight_dir, 'sam2_output/*.png')))
    for idx, frame in enumerate(data['frames_after_flight']):
        image = Image.open(frame['file_path']).convert('RGBA')
        sam2_mask = Image.open(sam2_output[idx]).convert('RGBA')

        combined_image = Image.blend(image, sam2_mask, 0.1)

        draw = ImageDraw.Draw(combined_image)
        
        for x, y in zip(frame['advected_x_coords'], frame['advected_y_coords']):
            draw.ellipse([(x-circle_radius, y-circle_radius),
                            (x+circle_radius, y+circle_radius)],
                            fill=(0, 255, 0), outline=(0, 255, 0))

        images.append(combined_image)

    # Save as .mp4
    output_file = os.path.join(output_video_dir, os.path.basename(meta_data_file).split('.')[0] + '.mp4')
    imageio.mimwrite(output_file, images, quality=8, fps=25)